# Generador de Rutas Óptimas

Este notebook contiene una función reutilizable para generar rutas óptimas entre dos puntos en cualquier ciudad usando algoritmo A* con penalizaciones.

## Importar librerías necesarias

In [18]:
import numpy as np
from shapely.geometry import LineString, Point
from math import sqrt
import osmnx as ox
import networkx as nx
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
import math
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Configuración de parámetros globales

In [29]:
# Parámetros de penalización
PENAL_SEMAFORO = 30      # segundos
FACTOR_ZONA_ESCOLAR = 1.2
PENAL_VIA_RESIDENCIAL = 1.1
DIST_SEMAFORO = 20       # metros
DIST_ESCUELA = 50        # metros

# Velocidades por tipo de vía
hwy_speeds = {
    "motorway": 80,
    "motorway_link": 70,
    "trunk": 80,
    "trunk_link": 70,
    "primary": 60,
    "primary_link": 50,
    "secondary": 50,
    "secondary_link": 45,
    "tertiary": 30,
    "unclassified": 30,
    "residential": 20,
    "service": 10,
}

# Penalizaciones por jerarquía de vía
jerarquia_penal = {
    "motorway":    0,
    "trunk":       0,
    "primary":     0,
    "secondary":   2,
    "tertiary":   20,
    "residential": 20,
    "service":     30,
}

In [20]:
# Sistema de caché para almacenar grafos de ciudades
# Esto evita descargar la misma ciudad múltiples veces
CACHE_CIUDADES = {}
CACHE_SEMAFOROS = {}
CACHE_ESCUELAS = {}

def limpiar_cache():
    """
    Limpia el caché de ciudades para liberar memoria.
    Útil si has cargado muchas ciudades y quieres liberar espacio.
    """
    global CACHE_CIUDADES, CACHE_SEMAFOROS, CACHE_ESCUELAS
    CACHE_CIUDADES.clear()
    CACHE_SEMAFOROS.clear()
    CACHE_ESCUELAS.clear()
    print("🧹 Caché limpiado exitosamente")

def mostrar_cache():
    """
    Muestra qué ciudades están actualmente en caché.
    """
    print("📋 Ciudades en caché:")
    if CACHE_CIUDADES:
        for ciudad in CACHE_CIUDADES.keys():
            print(f"  - {ciudad}")
    else:
        print("  (No hay ciudades en caché)")
    
    print(f"\n💾 Memoria utilizada: {len(CACHE_CIUDADES)} ciudades")

## Función principal para generar rutas

In [21]:
def generar_ruta_optima(inicio, destino, ciudad, mostrar_mapa=True, zoom=12, ancho=1000, alto=600, usar_cache=True):
    """
    Genera una ruta óptima entre dos puntos usando algoritmo A* con penalizaciones.
    
    Parámetros:
    - inicio (str): Dirección de inicio (ej: 'Sede Posgrados EIA, Envigado, Colombia')
    - destino (str): Dirección de destino (ej: 'Oficinas Almacenes Éxito, Envigado, Colombia')
    - ciudad (str): Ciudad donde buscar (ej: 'Envigado, Antioquia, Colombia')
    - mostrar_mapa (bool): Si mostrar el mapa interactivo
    - zoom (int): Nivel de zoom del mapa
    - ancho (int): Ancho del mapa en píxeles
    - alto (int): Alto del mapa en píxeles
    - usar_cache (bool): Si usar el caché para evitar descargar la misma ciudad múltiples veces
    
    Retorna:
    - dict: Diccionario con información de la ruta y figura del mapa
    """
    
    print(f"Procesando ruta de '{inicio}' a '{destino}' en {ciudad}...")
    
    try:
        # 1. Verificar si la ciudad está en caché
        if usar_cache and ciudad in CACHE_CIUDADES:
            print("💾 Usando ciudad desde caché (no se descarga nuevamente)...")
            G = CACHE_CIUDADES[ciudad]['grafo']
            G_proj = CACHE_CIUDADES[ciudad]['grafo_proj']
            signals_union = CACHE_SEMAFOROS.get(ciudad, None)
            schools_union = CACHE_ESCUELAS.get(ciudad, None)
        else:
            # 1a. Obtener el grafo de la ciudad
            print("📍 Descargando mapa de la ciudad...")
            G = ox.graph_from_place(ciudad, network_type='drive', simplify=False)
            
            # 2. Agregar velocidades y tiempos de viaje
            print("⚡ Calculando velocidades y tiempos...")
            G = ox.add_edge_speeds(G, hwy_speeds=hwy_speeds)
            G = ox.add_edge_travel_times(G)
            
            # 3. Proyectar el grafo
            G_proj = ox.project_graph(G)
            
            # 5. Descargar datos de semáforos y escuelas para penalizaciones
            print("🚦 Descargando semáforos y escuelas...")
            try:
                tags_ts = {'highway': 'traffic_signals'}
                gdf_ts = ox.features.features_from_place(ciudad, tags=tags_ts).to_crs(G_proj.graph['crs'])
                signals_union = gdf_ts.union_all()
            except:
                print("⚠️ No se encontraron semáforos, continuando sin penalizaciones por semáforos")
                signals_union = None
                
            try:
                tags_school = {'amenity': 'school'}
                gdf_school = ox.features.features_from_place(ciudad, tags=tags_school).to_crs(G_proj.graph['crs'])
                schools_union = gdf_school.union_all()
            except:
                print("⚠️ No se encontraron escuelas, continuando sin penalizaciones por zonas escolares")
                schools_union = None
            
            # Guardar en caché si está habilitado
            if usar_cache:
                print("💾 Guardando ciudad en caché para futuros usos...")
                CACHE_CIUDADES[ciudad] = {
                    'grafo': G,
                    'grafo_proj': G_proj
                }
                CACHE_SEMAFOROS[ciudad] = signals_union
                CACHE_ESCUELAS[ciudad] = schools_union
        
        # 4. Geocodificar direcciones con reintentos y timeout mejorado
        print("🔍 Buscando ubicaciones...")
        locator = Nominatim(user_agent='RutaOptimaGenerator', timeout=10)
        
        # Buscar inicio con reintentos
        location_start = None
        for intento in range(3):
            try:
                location_start = locator.geocode(inicio)
                if location_start:
                    break
                print(f"⚠️ Intento {intento + 1}: No se encontró '{inicio}', reintentando...")
            except Exception as e:
                print(f"⚠️ Error en intento {intento + 1} para inicio: {str(e)}")
                if intento < 2:
                    import time
                    time.sleep(2)  # Esperar 2 segundos antes del siguiente intento
        
        # Buscar destino con reintentos
        location_end = None
        for intento in range(3):
            try:
                location_end = locator.geocode(destino)
                if location_end:
                    break
                print(f"⚠️ Intento {intento + 1}: No se encontró '{destino}', reintentando...")
            except Exception as e:
                print(f"⚠️ Error en intento {intento + 1} para destino: {str(e)}")
                if intento < 2:
                    import time
                    time.sleep(2)  # Esperar 2 segundos antes del siguiente intento
        
        if not location_start:
            raise ValueError(f"No se pudo encontrar la ubicación: {inicio}")
        if not location_end:
            raise ValueError(f"No se pudo encontrar la ubicación: {destino}")
            
        print(f"✅ Inicio: {location_start.address}")
        print(f"✅ Destino: {location_end.address}")
        
        # 6. Calcular penalizaciones (solo si no están ya calculadas)
        need_penalties = any('travel_time_penalizado' not in data for _, _, _, data in G_proj.edges(keys=True, data=True))
        
        if need_penalties:
            print("🎯 Calculando penalizaciones...")
            for u, v, k, data in G_proj.edges(keys=True, data=True):
                # Crear geometría si no existe
                geom = data.get("geometry")
                if geom is None:
                    x1, y1 = G_proj.nodes[u]['x'], G_proj.nodes[u]['y']
                    x2, y2 = G_proj.nodes[v]['x'], G_proj.nodes[v]['y']
                    geom = LineString([(x1, y1), (x2, y2)])
                    data['geometry'] = geom
                
                # Calcular distancias a elementos de penalización
                if signals_union is not None:
                    data['signal_dist'] = geom.distance(signals_union)
                else:
                    data['signal_dist'] = math.inf
                    
                if schools_union is not None:
                    data['school_dist'] = geom.distance(schools_union)
                else:
                    data['school_dist'] = math.inf
                
                # Calcular tiempo penalizado
                base_tt = data.get("travel_time")
                if base_tt is None:
                    base_tt = data['length'] / (30/3.6)
                
                penal = base_tt
                
                # Penalizaciones por entorno
                if data.get("signal_dist", math.inf) <= DIST_SEMAFORO:
                    penal += 10
                if data.get("school_dist", math.inf) <= DIST_ESCUELA:
                    penal *= FACTOR_ZONA_ESCOLAR
                
                # Penalizaciones por jerarquía de vía
                hw = data.get("highway", "unclassified")
                if isinstance(hw, list):
                    hw = hw[0]
                penal += jerarquia_penal.get(hw, 15)
                
                data["travel_time_penalizado"] = penal
        else:
            print("✅ Penalizaciones ya calculadas (desde caché)")
        
        # 7. Crear función heurística
        all_speeds = [edata.get('speed_kph', 40) for _, _, _, edata in G_proj.edges(keys=True, data=True)]
        vmax = max(all_speeds)
        vmax_mps = vmax / 3.6
        
        def heuristica_tiempo(u, v):
            x1, y1 = G_proj.nodes[u]['x'], G_proj.nodes[u]['y']
            x2, y2 = G_proj.nodes[v]['x'], G_proj.nodes[v]['y']
            distancia = sqrt((x2 - x1)**2 + (y2 - y1)**2)
            return distancia / vmax_mps
        
        # 8. Obtener nodos más cercanos
        start = (location_start.latitude, location_start.longitude)
        end = (location_end.latitude, location_end.longitude)
        
        start_node = ox.distance.nearest_nodes(G, X=start[1], Y=start[0])
        end_node = ox.distance.nearest_nodes(G, X=end[1], Y=end[0])
        
        # 9. Calcular ruta óptima con A*
        print("🚀 Calculando ruta óptima con A*...")
        ruta_astar = nx.astar_path(G_proj, start_node, end_node,
                                   heuristic=lambda u, v=end_node: heuristica_tiempo(u, v),
                                   weight='travel_time_penalizado')
        
        # 10. Calcular métricas de la ruta
        tiempo_total = sum(G_proj.edges[u, v, 0]['travel_time_penalizado'] for u, v in zip(ruta_astar[:-1], ruta_astar[1:]))
        distancia_total = sum(G_proj.edges[u, v, 0]['length'] for u, v in zip(ruta_astar[:-1], ruta_astar[1:]))
        
        print(f"✅ Ruta calculada: {distancia_total:.0f} m, {tiempo_total:.1f} segundos")
        
        # 11. Preparar datos para visualización
        node_start, node_end = [], []
        X_to, Y_to, X_from, Y_from = [], [], [], []
        length, travel_time = [], []
        
        for u, v in zip(ruta_astar[:-1], ruta_astar[1:]):
            node_start.append(u)
            node_end.append(v)
            length.append(round(G.edges[(u, v, 0)]['length']))
            travel_time.append(round(G.edges[(u, v, 0)]['travel_time']))
            X_from.append(G.nodes[u]['x'])
            Y_from.append(G.nodes[u]['y'])
            X_to.append(G.nodes[v]['x'])
            Y_to.append(G.nodes[v]['y'])
        
        df_ruta = pd.DataFrame({
            "node_start": node_start,
            "node_end": node_end,
            "X_from": X_from,
            "Y_from": Y_from,
            "X_to": X_to,
            "Y_to": Y_to,
            "length": length,
            "travel_time": travel_time
        })
        
        # 12. Crear mapa interactivo (ACTUALIZADO: usando scattermap en lugar de scattermapbox)
        fig = None
        if mostrar_mapa and len(df_ruta) > 0:
            print("🗺️ Generando mapa interactivo...")
            
            # Crear figura base
            fig = go.Figure()
            
            # Agregar líneas de la ruta usando scattermap (nueva API)
            for i in range(len(df_ruta)):
                fig.add_trace(go.Scattermap(
                    mode="lines",
                    lon=[df_ruta.iloc[i]['X_from'], df_ruta.iloc[i]['X_to']],
                    lat=[df_ruta.iloc[i]['Y_from'], df_ruta.iloc[i]['Y_to']],
                    line=dict(width=4, color='red'),
                    showlegend=False,
                    hovertemplate=f"Segmento {i+1}<br>Distancia: {df_ruta.iloc[i]['length']}m<br>Tiempo: {df_ruta.iloc[i]['travel_time']}s<extra></extra>"
                ))
            
            # Agregar marcadores de inicio y fin usando scattermap
            fig.add_trace(go.Scattermap(
                mode="markers",
                lon=[start[1]],
                lat=[start[0]],
                marker=dict(size=15, color='green'),
                text=["Inicio"],
                name="Inicio",
                hovertemplate=f"<b>Inicio</b><br>{location_start.address}<extra></extra>"
            ))
            
            fig.add_trace(go.Scattermap(
                mode="markers",
                lon=[end[1]],
                lat=[end[0]],
                marker=dict(size=15, color='red'),
                text=["Destino"],
                name="Destino",
                hovertemplate=f"<b>Destino</b><br>{location_end.address}<extra></extra>"
            ))
            
            # Configurar el layout del mapa (actualizado para usar map en lugar de mapbox)
            center_lat = (start[0] + end[0]) / 2
            center_lon = (start[1] + end[1]) / 2
            
            fig.update_layout(
                map=dict(
                    style="open-street-map",
                    center=dict(lat=center_lat, lon=center_lon),
                    zoom=zoom
                ),
                width=ancho,
                height=alto,
                title=f"Ruta Óptima: {inicio} → {destino}",
                showlegend=True
            )
        
        # 13. Preparar resultado
        resultado = {
            'ruta_exitosa': True,
            'inicio': location_start.address,
            'destino': location_end.address,
            'ciudad': ciudad,
            'distancia_metros': round(distancia_total),
            'tiempo_segundos': round(tiempo_total, 1),
            'tiempo_minutos': round(tiempo_total / 60, 1),
            'nodos_ruta': ruta_astar,
            'dataframe_ruta': df_ruta,
            'figura_mapa': fig,
            'coordenadas_inicio': start,
            'coordenadas_destino': end,
            'usado_cache': usar_cache and ciudad in CACHE_CIUDADES
        }
        
        print(f"🎉 ¡Ruta generada exitosamente!")
        print(f"📏 Distancia total: {resultado['distancia_metros']} metros")
        print(f"⏱️ Tiempo estimado: {resultado['tiempo_minutos']} minutos")
        if usar_cache:
            print(f"💾 Caché: {'Usado' if resultado['usado_cache'] else 'Guardado para próxima vez'}")
        
        return resultado
        
    except Exception as e:
        print(f"❌ Error al generar la ruta: {str(e)}")
        return {
            'ruta_exitosa': False,
            'error': str(e),
            'inicio': inicio,
            'destino': destino,
            'ciudad': ciudad
        }

### ⚠️ Nota sobre actualización de Plotly

**Actualización importante**: Este código usa la nueva API de Plotly:
- ✅ `go.Scattermap()` (nueva API recomendada)
- ❌ `go.Scattermapbox()` (deprecada)

Esto elimina las advertencias de deprecación y asegura compatibilidad futura.

## Función auxiliar para mostrar solo el mapa

In [22]:
def mostrar_mapa_ruta(resultado):
    """
    Muestra solo el mapa de un resultado de ruta generado.
    
    Parámetros:
    - resultado (dict): Resultado de la función generar_ruta_optima
    """
    if resultado['ruta_exitosa'] and resultado['figura_mapa'] is not None:
        # Mostrar el mapa usando .show() que es la forma correcta en Jupyter
        resultado['figura_mapa'].show()
        return resultado['figura_mapa']
    else:
        print("No hay mapa disponible para mostrar")
        return None

## Función de ejemplo de uso

In [23]:
def ejemplo_uso():
    """
    Ejemplo de cómo usar las funciones para generar rutas.
    """
    # Ejemplo 1: Ruta en Envigado
    print("=== EJEMPLO 1: Ruta en Envigado ===")
    resultado1 = generar_ruta_optima(
        inicio="Sede Posgrados EIA, Envigado, Colombia",
        destino="Oficinas Almacenes Éxito, Envigado, Colombia",
        ciudad="Envigado, Antioquia, Colombia"
    )
    
    if resultado1['ruta_exitosa']:
        print(f"\n📊 Resumen de la ruta:")
        print(f"Desde: {resultado1['inicio']}")
        print(f"Hasta: {resultado1['destino']}")
        print(f"Distancia: {resultado1['distancia_metros']} metros")
        print(f"Tiempo: {resultado1['tiempo_minutos']} minutos")
        
        # Mostrar el mapa
        if resultado1['figura_mapa']:
            resultado1['figura_mapa'].show()
    
    return resultado1

## Ejecutar ejemplo

In [24]:
# Ejecutar el ejemplo
resultado_ejemplo = ejemplo_uso()

=== EJEMPLO 1: Ruta en Envigado ===
Procesando ruta de 'Sede Posgrados EIA, Envigado, Colombia' a 'Oficinas Almacenes Éxito, Envigado, Colombia' en Envigado, Antioquia, Colombia...
📍 Descargando mapa de la ciudad...
⚡ Calculando velocidades y tiempos...
⚡ Calculando velocidades y tiempos...
🚦 Descargando semáforos y escuelas...
🚦 Descargando semáforos y escuelas...
💾 Guardando ciudad en caché para futuros usos...
🔍 Buscando ubicaciones...
💾 Guardando ciudad en caché para futuros usos...
🔍 Buscando ubicaciones...
✅ Inicio: Sede Posgrados EIA, Calle 25 Sur, La Magnolia, Envigado, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 055422, Colombia
✅ Destino: Oficinas Almacenes Éxito, Avenida Carrera 48, La Magnolia, Envigado, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 055422, Colombia
🎯 Calculando penalizaciones...
✅ Inicio: Sede Posgrados EIA, Calle 25 Sur, La Magnolia, Envigado, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 055422, Colombia
✅ Destino: Oficinas A

## Uso personalizado

Puedes usar la función principal directamente así:

In [30]:
# Ejemplo de uso personalizado - Demostrando el sistema de caché
# IMPORTANTE: Ejecuta esta celda solo una vez para evitar duplicación

print("=== PRIMERA EJECUCIÓN (descarga la ciudad) ===")

# Definir ubicaciones
mi_inicio = "Carrera 84B, Loma de Los Bernal, Comuna 16 - Belén, Perímetro Urbano Medellín, Medellín, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 050026, Colombia"
mi_destino = "Unicentro, Medellín, Colombia"  # Simplificado para mejor geocodificación
mi_ciudad = "Medellín, Antioquia, Colombia"

# Generar la primera ruta
mi_resultado = generar_ruta_optima(
    inicio=mi_inicio,
    destino=mi_destino,
    ciudad=mi_ciudad,
    zoom=13,
    ancho=1200,
    alto=800
)

# Mostrar resultado de la primera ruta
if mi_resultado['ruta_exitosa']:
    print("\n🎯 Primera ruta generada:")
    print(f"📍 Origen: {mi_resultado['inicio']}")
    print(f"🏁 Destino: {mi_resultado['destino']}")
    print(f"📏 Distancia: {mi_resultado['distancia_metros']} metros")
    print(f"⏱️ Tiempo estimado: {mi_resultado['tiempo_minutos']} minutos")
    
    # Mostrar el mapa si se generó correctamente
    if mi_resultado['figura_mapa']:
        print("🗺️ Mostrando mapa de la primera ruta...")
        mi_resultado['figura_mapa'].show()
else:
    print(f"❌ Error en primera ruta: {mi_resultado['error']}")

print("\n" + "="*60)
print("=== SEGUNDA EJECUCIÓN (usa caché - más rápido) ===")

# Generar una segunda ruta diferente en la misma ciudad
mi_resultado2 = generar_ruta_optima(
    inicio="Universidad Nacional de Colombia, Medellín",  # Simplificado
    destino="Estación Universidad, Medellín",            # Simplificado
    ciudad=mi_ciudad  # Misma ciudad, debería usar caché
)

# Mostrar resultado de la segunda ruta
if mi_resultado2['ruta_exitosa']:
    print(f"\n🚀 Segunda ruta (desde caché):")
    print(f"📍 Origen: {mi_resultado2['inicio']}")
    print(f"🏁 Destino: {mi_resultado2['destino']}")
    print(f"📏 Distancia: {mi_resultado2['distancia_metros']} metros")
    print(f"⏱️ Tiempo estimado: {mi_resultado2['tiempo_minutos']} minutos")
    
    # Mostrar el mapa de la segunda ruta
    if mi_resultado2['figura_mapa']:
        print("🗺️ Mostrando mapa de la segunda ruta...")
        mi_resultado2['figura_mapa'].show()
else:
    print(f"❌ Error en segunda ruta: {mi_resultado2['error']}")

# Mostrar estado del caché
print("\n" + "="*40)
print("📊 ESTADO DEL CACHÉ:")
mostrar_cache()

=== PRIMERA EJECUCIÓN (descarga la ciudad) ===
Procesando ruta de 'Carrera 84B, Loma de Los Bernal, Comuna 16 - Belén, Perímetro Urbano Medellín, Medellín, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 050026, Colombia' a 'Unicentro, Medellín, Colombia' en Medellín, Antioquia, Colombia...
💾 Usando ciudad desde caché (no se descarga nuevamente)...
🔍 Buscando ubicaciones...
✅ Inicio: Carrera 84B, Loma de Los Bernal, Comuna 16 - Belén, Perímetro Urbano Medellín, Medellín, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 050026, Colombia
✅ Destino: Unicentro, Los Conquistadores, Comuna 11 - Laureles-Estadio, Perímetro Urbano Medellín, Medellín, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 050031, Colombia
✅ Inicio: Carrera 84B, Loma de Los Bernal, Comuna 16 - Belén, Perímetro Urbano Medellín, Medellín, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 050026, Colombia
✅ Destino: Unicentro, Los Conquistadores, Comuna 11 - Laureles-Estadio, Perímetro Urbano Mede


=== SEGUNDA EJECUCIÓN (usa caché - más rápido) ===
Procesando ruta de 'Universidad Nacional de Colombia, Medellín' a 'Estación Universidad, Medellín' en Medellín, Antioquia, Colombia...
💾 Usando ciudad desde caché (no se descarga nuevamente)...
🔍 Buscando ubicaciones...
✅ Inicio: Universidad Nacional de Colombia, Calle 58, Comuna 7 - Robledo, Perímetro Urbano Medellín, Medellín, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 050010, Colombia
✅ Destino: Estación EnCicla Universidad, Ciclorruta Jardin Botanico, San Pedro, Comuna 4 - Aranjuez, Perímetro Urbano Medellín, Medellín, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 050006, Colombia
✅ Inicio: Universidad Nacional de Colombia, Calle 58, Comuna 7 - Robledo, Perímetro Urbano Medellín, Medellín, Valle de Aburrá, Antioquia, RAP del Agua y la Montaña, 050010, Colombia
✅ Destino: Estación EnCicla Universidad, Ciclorruta Jardin Botanico, San Pedro, Comuna 4 - Aranjuez, Perímetro Urbano Medellín, Medellín, Valle de Aburr


📊 ESTADO DEL CACHÉ:
📋 Ciudades en caché:
  - Envigado, Antioquia, Colombia
  - Medellín, Antioquia, Colombia

💾 Memoria utilizada: 2 ciudades


## 💡 Consejos de uso y troubleshooting

### ⚠️ Problemas comunes y soluciones:

1. **Mensajes duplicados**: 
   - No ejecutes la misma celda múltiples veces rápidamente
   - Espera a que termine completamente antes de ejecutar nuevamente

2. **Error de conexión con Nominatim**:
   - El sistema ahora reintenta automáticamente hasta 3 veces
   - Si persiste, usa nombres de lugares más simples y conocidos

3. **El mapa no se muestra**:
   - Verifica que `resultado['ruta_exitosa']` sea `True`
   - Asegúrate de que tienes conexión a internet
   - La función ahora llama automáticamente a `.show()`

4. **Para mejor geocodificación, usa nombres como**:
   - ✅ "Universidad Nacional, Medellín"
   - ✅ "Planetario de Medellín"  
   - ❌ "Universidad Nacional de Colombia sede Medellín"

## Gestión del caché

El sistema de caché te permite gestionar qué ciudades mantener en memoria:

In [26]:
# Ejemplos de gestión del caché

# Ver qué ciudades están en caché
mostrar_cache()

# Limpiar todo el caché si necesitas liberar memoria
# limpiar_cache()

# Generar ruta sin usar caché (forzar descarga)
# resultado_sin_cache = generar_ruta_optima(
#     inicio="Punto A",
#     destino="Punto B", 
#     ciudad="Tu Ciudad",
#     usar_cache=False  # Esto fuerza la descarga
# )

📋 Ciudades en caché:
  - Envigado, Antioquia, Colombia
  - Medellín, Antioquia, Colombia

💾 Memoria utilizada: 2 ciudades
